# A class on Python classes

## Python Basics

Everything in Python is an object with an **identity** and **type**!

You can access them with `id()` and `type()`

Python already has built-in types (None, int, str, ...) which historically could not be extended

**In Python3 all types are also classes**, the terms are used interchangably and you can inherent from everything

### Terminology

Class declaration

In [ ]:
class MyClass:
    pass

Creating an instance of that class

In [ ]:
obj = MyClass()
print(type(obj))

What are the built-in types?

In [ ]:
age = 50
print(type(age))

So a integer declaration actually returns an instance of the int class

In [ ]:
age = int(50)

### How deep does the rabbit hole go?

Does `int` have a base class?

In [ ]:
print(int.__bases__)

Does `object` have a base class?

In [ ]:
print(object.__bases__)

The `object` class is the basis of all classes in Python3

In [ ]:
class MyClass:
    pass

# is equivalent to

class MyClass(object):
    pass

Everything in Python is an object!

If everything is an object, what is the type of a class declaration?

In [ ]:
print(type(int))
print(type(MyClass))
print(type(object))

Class declarations are instances of the class `type`, which itself is an instance of `type` and a subclass of the class `object`

In [ ]:
print(type(type))
print(type.__bases__)

`type` is a so-called **metaclass**, a class whose instances are classes

### Dynamic class declaration

instead of only one argument, type can be called with three parameters:

`type(classname, superclasses, attributes_dict)`

In [ ]:
MyClass = type('MyClass', (), {})
obj = MyClass()
print(type(obj))

## Back to something useful - how can I enhance my classes

- fundamentally classes are object constructors
- allow defining custom data objects, bundeling data and functionalities
- similar to structs/classes in c++ (difference is default public/private access)
- in python all members are public and all member functions are virtual

### Anatomy of a class

In [ ]:
class MyClass:
    """A simple example class."""
    unmutable = 100 # class attribute shared by each instance
    mutable = []    # be careful with mutable class attributes (avoid this!)
   
    def set_instance_attr(self, value): # instance method, despite the name not unique to each instance
        """Assign to an instance attribute"""
        self.instance_attr = value # instance attribute unique to each instance

# Create two instances of MyClass
obj_1 = MyClass()
obj_2 = MyClass()
# This example shows what goes wrong with mutable class attributes
obj_2.mutable.append('Obacht!')
print(obj_1.mutable)

### Attributes

- all attributes/instance attributes of a class/instance are stored in a dictionary

In [ ]:
print('Class attributes:', MyClass.__dict__)
print('Instance attributes:', obj_1.__dict__)

- dictionaries are memory heavy compared to lists
- if you do not need to add attributes to your instances dynamically, you can use `__slots__`
- `__slots__` attributes also have faster access O(1) vs. O(N)

In [ ]:
import sys

class MyClassWithSlots:
    """A simple example class using slots."""
    __slots__ = ['instance__attr']
    unmutable = 100
    mutable = []

    def set_instance_attr(self, value):
        """Assign to an instance attribute"""
        self.instance_attr = value

# Compare the size of classes, the one using slots is smaller 
obj_slots = MyClassWithSlots()
print('Size of standard class:', sys.getsizeof(obj_1))
print('Size of class using slots:', sys.getsizeof(obj_slots))
# However you loose the ability to assign instance attributes dynamically
obj_slots.new_attr = 42

- you can still allow dynamic instance attributes by adding `'__dict__'` to `__slots__`

### Private attributes and methods

- single underscore only indicates to others that the attribute/method is intended to be private (except wildcard imports)
- double underscore `__var` is textually replaced with `_classname__var` (name mangling)

In [ ]:
class MyClass:
    """An example class with private attributes and methods"""
    __private_attr = 'You see my private attribute!'
    
    def __private_method(self):
        """A private method"""
        print('You see my private method!')

obj = MyClass()
# You can still acces private attributes by using the mangled name
print(obj._MyClass__private_attr)
obj._MyClass__private_method()
# However, accessing it directly will give an error
print(obj.__private_attr)

## Some useful class method decorators

### @property decorator - how to convert a method to an attribute

Changing instance variables do not change other instance variables

In [ ]:
class Person:
    
    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname
        self.fullname = self.first + ' '+ self.last
    
person = Person('Reanu', 'Keeves')
print(person.fullname)
person.first = 'Bob'
print(person.fullname)

@property decorator allows to continue to use methods as attributes in order to not break existing code

In [ ]:
class Person:

    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname
    
    @property
    def fullname(self):
        return self.first + ' '+ self.last

person = Person('Reanu', 'Keeves')
print(person.fullname)
person.first = 'Bob'
print(person.fullname)

in addtion, it allows custom setting and deletion of attributes

In [ ]:
class Person:

    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname

    @property
    def fullname(self):
        return self.first + ' ' + self.last
     
    @fullname.setter
    def fullname(self, name):
        firstname, lastname = name.split()
        self.first = firstname
        self.last = lastname
        
    @fullname.deleter
    def fullname(self):
        self.first = 'None'
        self.last = 'None'  

In [ ]:
person = Person('Reanu', 'Keeves')
print(person.fullname)
# Set a new name
person.fullname = 'Elton Husk'
# Other attributes get automatically adjusted
print(person.fullname)
print(person.first)
print(person.last)
# Delete name
del person.fullname
print(person.fullname)

use properties for full encapsulation where you can define rules how private attributes can be got and set

In [ ]:
class Encapsulated:
    
    def __init__(self, name):
        self.private_attr = name # This assignment calls the setter method
        
    @property
    def private_attr(self):
        return self.__private_attr
    
    @private_attr.setter
    def private_attr(self, name):
        if len(name) > 0 and len(name) < 10:
            self.__private_attr = name
        else:
            self.__private_attr = None
        
obj = Encapsulated('Chris P Bacon')
# The name is too long
print(obj.private_attr)

### @staticmethod decorator - how to call methods without instantiating

- instance methods can only be called by instances of the class
- @staticmethod decorator converts it to a static method
- static methods can also be called without instantiating the class first
- static methods do not know about other attributes
- could use normal function, however this way it is logically contained in the class for readability

In [ ]:
class MyClass:
  
       @staticmethod
       def find_max(number):
           return max(number, 42)

# You can use the find_max method using the class directly
print(MyClass.find_max(101))
# or using an instance
obj = MyClass()
print(obj.find_max(101))

### @classmethod decorator - a static method which knows about its class

- @classmethod decorator converts an instance method to a class method
- similar to static methods thay can be called without instantiating the class first
- takes the class as parameter: `cls`
- class methods do know about the other attributes
- class methods allow predifined constructors (factory functions)

In [ ]:
class MyClass:
    other_number = 42
           
    @classmethod
    def find_max(cls, number):
        return max(number, cls.other_number)
    
print(MyClass.find_max(101))
obj = MyClass()
print(obj.find_max(101)) # This would also work with normal instance method

In [ ]:
class Pizza:
    """This class shows the concept of factory functions"""
    
    def __init__(self, ingredients):
        self.ingredients = ingredients

    def __repr__(self):
        return f'Pizza({self.ingredients})'

    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'tomatoes'])

    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])

Pizza.margherita()

## Special methods - adding functionality to your classes

### Example: What happens when we create an instance? 

In [ ]:
obj = MyClass()

- the `__call__` method of the parent class is called, in this case the `type_call` method of the `type` metaclass
- this in turn invokes the `__new__` and `__init__` methods of the class itself
- all of these type of methods are called **special**, **magic** or **dunder** methods and are pythons way of operator overloading

### `__new__` - modify how instances of the class are created

- automatically called when a class is instantiated
- actually creates the object
- used to customize how new class instances are created
- this is where immutable objects are initialized as any attribute set in `__init__` can be modified

In [ ]:
class Singleton:
    """This is an example of how to implement a singleton"""
    __instance = None
    
    def __new__(cls): # As in the case with self, cls is a naming convention
        if not cls.__instance:
            cls.__instance = object.__new__(cls)
            return cls.__instance
        else:
            print('An instance already exists!')
        
single_one = Singleton()
print(single_one)
single_two = Singleton()
print(single_two)

### `__init__` - the constructor/initilizer

- automatically called when a class is instantiated
- used to initialize attributes

### `__repr__` - the formal representation

- called by repr()
- should be able to act as an argument to eval() and return the same object
- should always be implemented

In [ ]:
class Person:
    
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __repr__(self):
        """Is called by the repr() function"""
        return (f'Person("{self.first}","{self.last}",{self.age})')

person = Person('Reanu', 'Keeves', 56)
# This should return itself
eval(repr(person))

### `__str__` - the string representation

- called by str() and the print statement
- is the human readable representation
- if not implemented, str() returns the result of the `__repr__` method
- the latter also always happens for objects in containers

In [ ]:
class Person:
    
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __repr__(self):
        return f'Person("{self.first}","{self.last}",{self.age})'

    def __str__(self):
        """Is called by the str() and print() function"""
        return f'{self.first} {self.last}, {self.age}'
    
person = Person('Reanu', 'Keeves', 56)
# Print a nice human readable version
print(person)

### `__del__` - the destructor/finilizer

- called when the garbage collector is deleting objects without references

In [ ]:
class MyClass:
    
    def __del__(self):
        """Is called when the garbage collector frees up the memory"""
        print(f'{self.__class__.__name__} class destroyed!')
        
    def __str__(self):
        return "I exist!"
        
obj = MyClass()
reference = obj
print(obj)
del obj
print(reference)
del reference

- be aware that garbage collection might not happen at all
- therefore prefere **try/finally** or **with** for critical code or call `__del__` directly

In [ ]:
class MyClass:

    def __enter__(self):
        """Gets called when with statement is entered"""
        print('Entering with statement!')
        
    def __exit__(self, exc_type, exc_value, traceback):
        """Gets called when with statement is left"""
        print('This is always garanteed to be called!')

with MyClass() as example:
    print('Executing code!')

### `__getattr__`, `__setattr__`, `__delattr__` - you should probably be using a property

In [ ]:
class MyClass:
    
    def __setattr__(self, name, value):
        """Gets called when an attribute is set"""
        print('Attribute setting forbidden!')
        
    def __getattribute__(self, name):
        """Gets called when any attribute is accesses"""
        print('The answer is always 42!')
        raise AttributeError
        
    def __getattr__(self, name):
        """Gets called when an attribute error is raised"""
        print('I told you the answer is always 42!')
        
    def __delattr__(self, name):
        """Gets called when an attribute is deleted"""
        print('You cannot suppress the truth!')
        
obj = MyClass()
# Try to set attribute
obj.x = 3
# Try to get attribute
obj.x
# Try to delete attribute
del obj.x
# If only __setattr__ is defined you can still instance dictionary directly
# This does not call the __setattr__ method: obj.__dict__['x'] = 3
# However, accessing the dictionary calls the __getattribute__ method

avoid infinite recursions, instead of

In [ ]:
class MyClass:
    
    def __setattr__(self, name, value):
        setattr(self, name, value)

use the respective method of the parent class

In [ ]:
class MyClass:
    
    def __setattr__(self, name, value):
        super().__setattr__(name, value)

### `__getitem__`, `__setitem__`, `__len__` - making your class a container

In [ ]:
class MyContainer:
    
    def __init__(self):
        self.data = []
        
    def __len__(self):
        return len(self.data)
    
    def append(self, item):
        self.data.append(item)
    
    def __getitem__(self, sliced):
        return self.data[sliced]
    
    def __setitem__(self, key, item):
        self.data[key] = item
        
container = MyContainer()
container.append("First")
container.append("Second")
container.append("Third")
print('First entry:', container[0])
print('Length:', len(container))
container[1] = 2
print('Second entry', container[1])

### `__eq__`, `__le__`, `__lt__`, `__ne__`, `__ge__`, `__gt__` -  comparing two objects

- so-called “rich comparison” methods
- default `__eq__` compares by using `is`
- only have to implement one of each kind

In [ ]:
class Person:
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __le__(self, other):
        return self.age <= other.age

    def __lt__(self, other):
        return self.age < other.age
    
person1 = Person('Reanu', 'Keeves', 56)
person2 = Person('Elton', 'Husk', 62)
print(person1 != person2)
print(person1 <= person2)
print(person1 > person2)

- using `total_ordering` you only have to implement one of `__lt__`, `__le__`, `__gt__` or `__ge__`
- if you do not implement `__eq__` the objects will still be only compared using `is`

In [ ]:
from functools import total_ordering

@total_ordering
class Person:
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __lt__(self, other):
        return self.age < other.age
    
    def __repr__(self):
        return f'Person("{self.first}","{self.last}",{self.age})'
    
person1 = Person('Reanu', 'Keeves', 56)
person2 = Person('Elton', 'Husk', 62)
print(person1 != person2)
print(person1 <= person2)
print(person1 < person2)

### `__hash__` - making your class hashable

if `__eq__` is not defined:
- every seperate instance will have a different hash
- only limited usage as hash, as objects created with same values have different hashes
- you should also not define a `__hash__` method

if `__eq__` is defined but not `__hash__`:
- instances will not be usable as items in hashable collections

if a class defines mutable objects, it should not implement `__hash__`, **only immutable objects should be used as hash**

In [ ]:
# Our previously defined persons cannot be used as keys in dictionaries
dic = {person1: 1, person2: 2}

In [ ]:
class Person:
    def __init__(self, firstname, lastname, age):
        self.first = firstname
        self.last = lastname
        self.age = age
    
    def __eq__(self, other):
        return self.age == other.age
    
    def __repr__(self):
        return f'Person("{self.first}","{self.last}",{self.age})'

    def __hash__(self):
        """The objects of the class are mutable!"""
        return hash(tuple([self.first, self.last, self.age]))

person1 = Person('Reanu', 'Keeves', 56)
person2 = Person('Reanu', 'Keeves', 56)
print(hash(person1) == hash(person2))
dic = {person1: 1}
# Everything seems to work, person2 is an accepted key 
print('dic[person2]:', dic[person2])
# We change person1 and both person1 and person2 will give a key error
person1.age = 49
dic[person2]

- when we changed person1, we also changed the dictionary key
- the hash will no longer refer to the correct bucket in the dictionary for that key
- only way to use mutable class is when the hash is based on the identity and not the value

## Dataclasses

- the `@dataclass` decorator converts a class to a dataclass
- automatically adds `__init__`, `__repr__`, and `__eq__`
- still functions as a regular class

In [ ]:
class Person:

    def __init__(self, firstname: str='unknown', lastname: str='unknown', age: int=-1): # These are variabel annotations or type hints
        self.first = firstname
        self.last = lastname
        self.age = age
        
    def __repr__(self):
        return f'Person(first={self.first}, last={self.last}, age={self.age})'
    
    def __eq__(self, other):
        return (self.first, self.last, self.age) == (other.first, other.last, other.age)

In [ ]:
from dataclasses import dataclass

@dataclass
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1

person1 = DataClassPerson('Reanu', 'Keeves', 56)
person2 = DataClassPerson('Reanu', 'Keeves', 56)
print(person1)
person1 == person2

type hints are mandatory for dataclasses, `any` is also an option 

- the lines below the class declaration define field objects, which are special to dataclasses and function like regular attributes
- mutable types are not allowed, except created by the `default_factory` option

In [ ]:
from dataclasses import dataclass, field

def return_list() -> list:
    return [0, 1, 2]

@dataclass
class DataClassMutable:
    mutable: list = field(default_factory=return_list)
        
obj = DataClassMutable()
print(obj.mutable)

fields have the following options:

- default: default value of the field
- default_factory: function that returns the initial value of the field
- init: use field in `__init__` method, default is True
- repr: use field in repr of the object, default is True
- compare: include the field in comparisons, default is True
- hash: include the field when calculating hash, default is to use the same as for compare
- metadata: a mapping with information about the field


In [ ]:
from dataclasses import dataclass, field, fields

@dataclass
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = field(default=-1, metadata={'unit': 'years'})
        
fields(DataClassPerson)

`@dataclass` decorator can also be called with the following parameters:

- init: adds `__init__` method, default is True
- repr: adds `__repr__` method, default is True
- eq:   adds `__eq__` method, default is True
- order: adds ordering methods, default is False
- unsafe_hash: force the addition of a `__hash__` method, default is False
- frozen: if True, assigning to fields (**and to fields only**) raises an exception, default is False

`order=True` compares objects as if there were tuples

In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1
                                   
person1 = DataClassPerson('Reanu', 'Keeves', 56)
person2 = DataClassPerson('Elton', 'Husk', 62)
person1 > person2

you have to set a sort index to get custom ordering

In [ ]:
from dataclasses import dataclass, field

@dataclass(order=True)
class DataClassPerson:
    sort_index: int = field(init=False, repr=False)
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1

    def __post_init__(self): # This is a special method of dataclasses, called by __init__
        self.sort_index = self.age

person1 = DataClassPerson('Reanu', 'Keeves', 56)
person2 = DataClassPerson('Elton', 'Husk', 62)
person1 > person2

`frozen=True` protects attributes from changing

In [ ]:
@dataclass(frozen=True)
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1
           
person1 = DataClassPerson('Reanu', 'Keeves', 56)
person1.first = 'Bob'

however, mutable objects can still be changed

In [ ]:
def return_list():
    return [0, 1, 2]

@dataclass(frozen=True)
class DataClassPerson:
    first: str = 'unknown'
    last: str = 'unknown'
    age: int = -1
    mutable: list = field(default_factory=return_list)
        
person1 = DataClassPerson('Reanu', 'Keeves', 56)
person1.mutable[0] = 42
print(person1.mutable)

## Conclusions

- everything in python is an object
- use **properties** for better attribute setting/getting and together with private variables for full encapsulation
- **special methods** can greatly expand the usability of classes; only implement as many as you need
- **dataclasses** can greatly ficilitate class creation